In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-robotics-er-1.5-preview")

## 1. StrOutputParser
The simplest parser - converts the model output to a plain string.

In [2]:
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chain = prompt | model | StrOutputParser()

response = chain.invoke({"topic": "programming"})
print(response)
print(type(response))  # <class 'str'>

Why did the programmer break up with the developer?

Because he found a bug in their relationship.

***

*Alternative, classic short joke:*

There are only 10 types of people in the world: those who understand binary, and those who don't.
<class 'str'>


## 2. JsonOutputParser
Parses the model output as JSON. Useful when you need structured data.

In [3]:
from langchain_core.output_parsers import JsonOutputParser

prompt = ChatPromptTemplate.from_template(
    """Generate information about a person with the following attributes:
    - name
    - age
    - occupation
    
    Topic: {topic}
    
    Return ONLY valid JSON, no other text."""
)

chain = prompt | model | JsonOutputParser()
response = chain.invoke({"topic": "a software developer"})
print(response)
print(type(response))  # <class 'dict'>

{'name': 'Alex Johnson', 'age': 35, 'occupation': 'Software Developer'}
<class 'dict'>


## 3. PydanticOutputParser
Parses output into a Pydantic model with validation. Best for type-safe structured data.

In [4]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Define Pydantic model
class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    director: str = Field(description="The director of the movie")
    year: int = Field(description="The release year")
    genres: List[str] = Field(description="List of genres")

parser = PydanticOutputParser(pydantic_object=Movie)

prompt = ChatPromptTemplate.from_template(
    """Generate information about a famous movie.
    
    {format_instructions}
    
    Movie topic: {topic}"""
)

chain = prompt | model | parser
response = chain.invoke({
    "topic": "a sci-fi movie from the 2000s",
    "format_instructions": parser.get_format_instructions()
})

print(response)
print(type(response))  # <class 'Movie'>
print(f"Title: {response.title}, Year: {response.year}")

title='Avatar' director='James Cameron' year=2009 genres=['Science Fiction', 'Action', 'Adventure']
<class '__main__.Movie'>
Title: Avatar, Year: 2009


## 4. CommaSeparatedListOutputParser
Parses comma-separated values into a Python list.

In [5]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

prompt = ChatPromptTemplate.from_template(
    """List 5 {topic}.
    
    {format_instructions}"""
)

chain = prompt | model | parser
response = chain.invoke({
    "topic": "programming languages",
    "format_instructions": parser.get_format_instructions()
})

print(response)
print(type(response))  # <class 'list'>
print(f"First item: {response[0]}")

['Python', 'JavaScript', 'Java', 'C++', 'C#']
<class 'list'>
First item: Python
